In [ ]:
import pandas as pd
import datetime as dt
import yfinance as yf
import json
from stockstats import wrap 


# get watch_list

In [ ]:
with open('watch_list.json') as f:
    watch_list = json.load(f)

# fetch data from yfinance api

In [ ]:
for symbol in watch_list:
    tk = yf.Ticker(f"{symbol}.BK")
    df = tk.history(period='6mo', interval='1d')
    df.to_csv(f"stock_data/{symbol}.csv")



# apply ta to data

In [ ]:
for symbol in watch_list:
    df = pd.read_csv(f"stock_data\{symbol}.csv")
    df_ta = wrap(df)
    df_ta.init_all()
    df.to_csv(f"stock_data_with_indicator\{symbol}.csv")

# show result with plotly

In [ ]:
import random
import plotly.graph_objects as go
symbol = random.choice(watch_list)
df = pd.read_csv(f"stock_data_with_indicator\{symbol}.csv")
fig = go.Figure(data=[go.Candlestick(x=df.index,
                                     open=df['open'],
                                     high=df['high'],
                                     low=df['low'],
                                     close=df['close'],
                                     name=symbol)])

fig.add_trace(go.Scatter(
    x=df.index, y=df["middle"], line_shape='spline', name="Middle"))
fig.add_trace(go.Scatter(
    x=df.index, y=df["supertrend"], line_shape='spline', name="supertrend"))
fig.add_trace(go.Scatter(
    x=df.index, y=df["boll"], line_shape='spline', name="boll"))
fig.add_trace(go.Scatter(
    x=df.index, y=df["boll_ub"], line_shape='spline', name="boll_ub"))
fig.add_trace(go.Scatter(
    x=df.index, y=df["boll_lb"], line_shape='spline', name="boll_lb"))

# fig name
fig.update_layout(
    title=f"{symbol} [ open = {df['open'].iloc[-1]:.2f} THB || high = {df['high'].iloc[-1]:.2f} THB || low = {df['low'].iloc[-1]:.2f} THB || close = {df['close'].iloc[-1]:.2f} THB ]")
# hide bottom range slider
fig.update_layout(xaxis_rangeslider_visible=False)

fig.show()
